In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from deep_translator import GoogleTranslator
import re
import datetime
import torch
import transformers
import tensorflow as tf
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')
# model = SentenceTransformer('all-MiniLM-L6-v2')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

f:\Python 3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel(r"C:\Users\50510\Desktop\YouTube_Projects\YouTube_Data.xlsx")
data.head()

,Date_Published,Title,Tags,channel,Comments,viewCount,likeCount,favoriteCount
0,2023-08-22 02:30:05+00:00,Naa Anveshana meet up in ￼Montreal and Vancouver Canada,"['Naa Anveshana meet up in Montreal and Vancouver Canada', 'Naaanveshana', 'naa anveshana', 'naa anveshana fight', 'naa anveshana girlfriend', 'naa anveshana latest troll', 'naa anveshana roast', 'naa anveshana telugu traveller', 'naa anveshana tissue', 'naa anveshana troll', 'naa anveshana trolls', 'naa anveshana vlogs', 'naa anveshana vs ravi telugu traveller', 'naa anveshana vs uma telugu traveller', 'naa anveshana youtube channel', 'naaanveshana', 'ravi telugu traveller vs naa anveshana']",Naa Anveshana,1169,333229,19698,0
1,2023-08-19 04:52:05+00:00,English Bay Beach Vancouver | Suspension bridge | Canada Place,"['Canada Place', 'English Bay Beach Vancouver', 'Naaanveshana', 'Suspension bridge', 'capilano suspension bridge', 'english bay beach', 'english bay beach walk', 'english bay vancouver', 'living in vancouver', 'moving to vancouver', 'vancouver', 'vancouver 4k', 'vancouver beach walk', 'vancouver canada', 'vancouver canada nightlife', 'vancouver downtown', 'vancouver english bay', 'vancouver english bay beach walk', 'vancouver island', 'vancouver seawall', 'vancouver summer beach', 'vancouver vlog', 'vancouver walk']",Naa Anveshana,2006,980291,51011,0
2,2023-08-17 03:47:03+00:00,48 hrs On Canada's Most Luxury train journey Banff to Vancouver,"['Naaanveshana', 'banff', 'british columbia', 'first passage to the west', 'luxury train canada', 'rocky mountaineer', 'rocky mountaineer 2023', 'rocky mountaineer experience', 'rocky mountaineer gold leaf', 'rocky mountaineer goldleaf', 'rocky mountaineer goldleaf service', 'rocky mountaineer jasper to vancouver', 'rocky mountaineer silverleaf', 'rocky mountaineer train', 'rocky mountaineer train ride canada', 'rocky mountaineer train trip', 'rocky mountaineer train trips', 'rocky mountaineer vancouver to banff']",Naa Anveshana,3774,1686573,74544,0
3,2023-08-15 02:30:31+00:00,Sulphur banff gondola | things to do in banff national park,"['Naaanveshana', 'Sulphur banff gondola', 'banff alberta', 'banff canada', 'banff gondola', 'banff gondola experience', 'banff gondola restaurant', 'banff gondola ride', 'banff gondola summer', 'banff gondola tickets', 'banff gondola to sulphur mountain alberta canada', 'banff gondola vlog', 'banff national park', 'best things to do in banff national park', 'gondola in banff', 'sulphur mountain banff', 'sulphur mountain gondola', 'things to do in banff canada', 'things to do in banff national park']",Naa Anveshana,2462,2189913,63843,0
4,2023-08-12 12:49:37+00:00,Columbia ice field Glacier Adventure Jasper National Park,"['Naaanveshana', 'athabasca falls', 'athabasca glacier', 'athabasca glacier tour', 'banff national park', 'best things to do in jasper', 'columbia icefield', 'hikes in jasper national park', 'icefields parkway', 'jasper', 'jasper canada 4k', 'jasper national park best', 'jasper national park hotels', 'jasper national park vlog', 'jasper skytram', 'maligne canyon', 'maligne lake', 'spirit island', 'sunwapta falls', 'things to do in jasper', 'things to do in jasper national park', 'what to do in jasper national park']",Naa Anveshana,3430,1481449,64606,0


In [3]:
## Cleaning Text

import re
#### Clean Text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[----------=]', " ", text)
    text = re.sub('       +', " ", text)  ### For removing long spaces
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', " ", text, flags=re.MULTILINE)

    # To Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = " ".join(text.split())  # Removing Extra Space
    return text

data['Cl_Text'] = data['Title'].apply(lambda x: clean_text(x))

# Remove system Generated letters
from string import ascii_letters
data['Cl_Text'] = data['Cl_Text'].map(lambda x: x.lstrip('+-').rstrip(ascii_letters))

In [4]:
# There are Some Telugu Words So we are Translating that into English


from deep_translator import GoogleTranslator
## ENGLISH TRANSLATION
def translator(text):
    text = GoogleTranslator(source='auto', target='en').translate(text)
    print(text)
    return text

data['English_Text'] = data['Title'].apply(lambda x: translator(x))

Naa Anveshana meet up in ￼Montreal and Vancouver Canada
English Bay Beach Vancouver | Suspension bridge | Canada Place
48 hrs On Canada's Most Luxury train journey Banff to Vancouver
Sulphur banff gondola | things to do in banff national park
Columbia ice field Glacier Adventure Jasper National Park
Banff National Park Lake Louise And Moraine lake
Montreal to Banff National Park Travel Journey
Naa Anveshana is No 1 travel influencer in India
Best Places to Visit in Montreal Canada
Best Things To Do In Ottawa Canada
Naa Anveshana Live From Canada
Toronto to ottawa train journey with via railway
Things To Do in Toronto Canada | Toronto island Beach
Naa anveshana meet up in Toronto Canada
Naaanveshana The World Traveller Live
Places To Visit in Niagara Falls Canada
30 Khalistan activists came to beat | Khalistan supporters in Canada
Hongkong to Canada Flight Journey | Cheapest Flight Tickets 2023
What should India learn from China? China Vs India Comparison
Last video in China | Last Day 

In [5]:
import spacy
import spacy_transformers
nlp = spacy.load('en_core_web_trf')
data['English_Text'] = data['English_Text'].apply(str)

# Write a code for punctuations

def punctuations(texts):
    doc = nlp(texts)
    tokenss = [token.text for token in doc if not token.is_punct]
    return " ".join(tokenss)

# Remove the Special Characters
def text_special(text):
    text = text.lower()
    text = text.replace('^', '')  
    text = text.replace('$', '') 
    text = text.replace('₹', '') 
    text = text.replace('+', '')
    text = text.replace('=', '')
    text = text.replace('ஃ', '')
    return " ".join(text.split())

data['Text'] = data['English_Text'].apply(lambda x: punctuations(x))
data['Text'] = data['Text'].apply(lambda x: text_special(x))

#### Find the Genre


For the Genre Prediction I used one personal model But i trained that model with different contexts is entirly different, So the prediction May not be Appropriate 100%,

In [7]:
encode_model = SentenceTransformer('bert-base-uncased')
encode_model.max_seq_length = 512
print("Max Sequence Length:", encode_model.max_seq_length)

data['Text'] = data['Text'].apply(str)

sentence_embeddings_data = encode_model.encode(data['Text'])
print("sentence_embeddings_Hindu done")

### Loading Model for prediction of Genre
Ge_model = tf.keras.models.load_model((r"C:\Users\Desktop\Model genre prediction\Genre_cls_retrained_model_KFold"),
                                       compile=False, 
                                       custom_objects={'KerasLayer':hub.KerasLayer})

Genre_prob = Ge_model.predict(sentence_embeddings_data)

Results = []
for i in Genre_prob:
    res = np.argmax(i)
    Results.append(res)

Genre_Results = pd.DataFrame(Results)
Genre_Results = Genre_Results.replace({0:'Business',
                           1: 'Crime',
                           2: 'Health&Nature',
                           3: 'Movie',
                           4: 'Spirutual',
                           5: 'National',
                           6: 'Sports',
                           7: 'Technology',
                           8: 'World',
                           9: 'National'})
data['Genre'] = Genre_Results[0]  # Take the data with Highest Scores

No sentence-transformers model found with name C:\Users\50510/.cache\torch\sentence_transformers\bert-base-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\50510/.cache\torch\sentence_transformers\bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifi

Max Sequence Length: 512
sentence_embeddings_Hindu done
71/71 [==============================] - 0s 1ms/step


In [8]:
data.head()

,Date_Published,Title,Tags,channel,Comments,viewCount,likeCount,favoriteCount,Cl_Text,English_Text,Text,Genre
0,2023-08-22 02:30:05+00:00,Naa Anveshana meet up in ￼Montreal and Vancouver Canada,"['Naa Anveshana meet up in Montreal and Vancouver Canada', 'Naaanveshana', 'naa anveshana', 'naa anveshana fight', 'naa anveshana girlfriend', 'naa anveshana latest troll', 'naa anveshana roast', 'naa anveshana telugu traveller', 'naa anveshana tissue', 'naa anveshana troll', 'naa anveshana trolls', 'naa anveshana vlogs', 'naa anveshana vs ravi telugu traveller', 'naa anveshana vs uma telugu traveller', 'naa anveshana youtube channel', 'naaanveshana', 'ravi telugu traveller vs naa anveshana']",Naa Anveshana,1169,333229,19698,0,naa anveshana meet up in montreal and vancouver,Naa Anveshana meet up in ￼Montreal and Vancouver Canada,naa anveshana meet up in ￼ montreal and vancouver canada,National
1,2023-08-19 04:52:05+00:00,English Bay Beach Vancouver | Suspension bridge | Canada Place,"['Canada Place', 'English Bay Beach Vancouver', 'Naaanveshana', 'Suspension bridge', 'capilano suspension bridge', 'english bay beach', 'english bay beach walk', 'english bay vancouver', 'living in vancouver', 'moving to vancouver', 'vancouver', 'vancouver 4k', 'vancouver beach walk', 'vancouver canada', 'vancouver canada nightlife', 'vancouver downtown', 'vancouver english bay', 'vancouver english bay beach walk', 'vancouver island', 'vancouver seawall', 'vancouver summer beach', 'vancouver vlog', 'vancouver walk']",Naa Anveshana,2006,980291,51011,0,english bay beach vancouver | suspension bridge | canada,English Bay Beach Vancouver | Suspension bridge | Canada Place,english bay beach vancouver | suspension bridge | canada place,World
2,2023-08-17 03:47:03+00:00,48 hrs On Canada's Most Luxury train journey Banff to Vancouver,"['Naaanveshana', 'banff', 'british columbia', 'first passage to the west', 'luxury train canada', 'rocky mountaineer', 'rocky mountaineer 2023', 'rocky mountaineer experience', 'rocky mountaineer gold leaf', 'rocky mountaineer goldleaf', 'rocky mountaineer goldleaf service', 'rocky mountaineer jasper to vancouver', 'rocky mountaineer silverleaf', 'rocky mountaineer train', 'rocky mountaineer train ride canada', 'rocky mountaineer train trip', 'rocky mountaineer train trips', 'rocky mountaineer vancouver to banff']",Naa Anveshana,3774,1686573,74544,0,48 hrs on canada's most luxury train journey banff to,48 hrs On Canada's Most Luxury train journey Banff to Vancouver,48 hrs on canada 's most luxury train journey banff to vancouver,World
3,2023-08-15 02:30:31+00:00,Sulphur banff gondola | things to do in banff national park,"['Naaanveshana', 'Sulphur banff gondola', 'banff alberta', 'banff canada', 'banff gondola', 'banff gondola experience', 'banff gondola restaurant', 'banff gondola ride', 'banff gondola summer', 'banff gondola tickets', 'banff gondola to sulphur mountain alberta canada', 'banff gondola vlog', 'banff national park', 'best things to do in banff national park', 'gondola in banff', 'sulphur mountain banff', 'sulphur mountain gondola', 'things to do in banff canada', 'things to do in banff national park']",Naa Anveshana,2462,2189913,63843,0,sulphur banff gondola | things to do in banff national,Sulphur banff gondola | things to do in banff national park,sulphur banff gondola | things to do in banff national park,Health&Nature
4,2023-08-12 12:49:37+00:00,Columbia ice field Glacier Adventure Jasper National Park,"['Naaanveshana', 'athabasca falls', 'athabasca glacier', 'athabasca glacier tour', 'banff national park', 'best things to do in jasper', 'columbia icefield', 'hikes in jasper national park', 'icefields parkway', 'jasper', 'jasper canada 4k', 'jasper national park best', 'jasper national park hotels', 'jasper national park vlog', 'jasper skytram', 'maligne canyon', 'maligne lake', 'spirit island', 'sunwapta falls', 'things to do in jasper', 'things to do in jasper national park', 'what to do in jasper national park'

#### Entity Extarction

In [9]:
import spacy
model = spacy.load(r"C:\Users\Desktop\Spacy_Language_models\GPU_Albert_Transformer_model_last\model-best")  # Custom Model

from collections import defaultdict, Counter

def ner_gpe(text, ent="GPE"):
    doc = model(text)
    return [X.text for X in doc.ents if X.label_ == ent]


def ner_person(text, ent="PERSON"):
    doc = model(text)
    return [X.text for X in doc.ents if X.label_ == ent]


def ner_norp(text, ent="NORP"):
    doc = model(text)
    return [X.text for X in doc.ents if X.label_ == ent]


def ner_loc(text, ent="LOC"):
    doc = model(text)
    return [X.text for X in doc.ents if X.label_ == ent]

data['Text'] = data['Text'].apply(str)

data['personalities_ents'] = data['Text'].apply(lambda x: ner_person(x))
data['Geographical_locations'] = data['Text'].apply(lambda x: ner_gpe(x))
data['Nationalities'] = data['Text'].apply(lambda x: ner_norp(x))
data['Locations'] = data['Text'].apply(lambda x: ner_loc(x))

In [11]:
## Replace Unwanted characters in columns
import re

def replace_squares(text):
    text = text.replace('[]', ' ')
    text = text.replace('[', ' ')
    text = text.replace(']', ' ')
    text = text.replace("'", " ")
    text = re.sub('       +', " ", text)  ### For removing long spaces
    return " ".join(text.split())

data['personalities_ents'] = data['personalities_ents'].apply(str)
data['Geographical_locations'] = data['Geographical_locations'].apply(str)
data['Nationalities'] = data['Nationalities'].apply(str)
data['Locations'] = data['Locations'].apply(str)

data['personalities_ents'] = data['personalities_ents'].apply(lambda x: replace_squares(x))
data['Geographical_locations'] = data['Geographical_locations'].apply(lambda x: replace_squares(x))
data['Nationalities'] = data['Nationalities'].apply(lambda x: replace_squares(x))
data['Locations'] = data['Locations'].apply(lambda x: replace_squares(x))

In [13]:
data.head(1)

,Date_Published,Title,Tags,channel,Comments,viewCount,likeCount,favoriteCount,Cl_Text,English_Text,Text,Genre,personalities_ents,Geographical_locations,Nationalities,Locations
0,2023-08-22 02:30:05+00:00,Naa Anveshana meet up in ￼Montreal and Vancouver Canada,"['Naa Anveshana meet up in Montreal and Vancouver Canada', 'Naaanveshana', 'naa anveshana', 'naa anveshana fight', 'naa anveshana girlfriend', 'naa anveshana latest troll', 'naa anveshana roast', 'naa anveshana telugu traveller', 'naa anveshana tissue', 'naa anveshana troll', 'naa anveshana trolls', 'naa anveshana vlogs', 'naa anveshana vs ravi telugu traveller', 'naa anveshana vs uma telugu traveller', 'naa anveshana youtube channel', 'naaanveshana', 'ravi telugu traveller vs naa anveshana']",Naa Anveshana,1169,333229,19698,0,naa anveshana meet up in montreal and vancouver,Naa Anveshana meet up in ￼Montreal and Vancouver Canada,naa anveshana meet up in ￼ montreal and vancouver canada,National,naa anveshana,"montreal , vancouver , canada",,


In [14]:
hi = pd.ExcelWriter('Cleaned_YouTube_Data.xlsx')
data.to_excel(hi)
hi.book.save('Cleaned_YouTube_Data.xlsx')